In [4]:
# imports
import requests
import os
import pandas as pd
import json  


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [6]:
# Assign Environment Variables
client_id = os.environ["FOURSQUARE_CLIENT_ID"]
client_secret = os.environ["FOURSQUARE_SECRET_CLIENT_ID"]
api_key = os.environ["FOURSQUARE_API_KEY"]

# Define the endpoint URL
url = "https://api.foursquare.com/v3/places/search"

# Create an empty list to store the results
results_restaurant = []
results_bar = []
results_art = []

# import city_bikes list of latitude and longitude
bike_stations_toronto = pd.read_csv(r'C:\Users\akans\Documents\Kierans_Documents\LHL\Statistical-Modelling-Project\data\bike_stations_toronto.csv')

# Define Headers for the API requests
headers = {
"Accept": "application/json",
"Authorization": api_key
}   

# Iterate through the DataFrame rows
for index, row in bike_stations_toronto.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    # Define parameters for the API request for this location query: restaurant
    params_restaurant = {
        'll': f'{latitude},{longitude}',
        'radius': 1000,
        'query': 'restaurant'
    }

    # Get requests for params_restaurant
    response_restaurant = requests.get(url, params=params_restaurant, headers=headers)

    # Check the response status code and add values to results list if exists
    if response_restaurant.status_code == 200:
        try: 
            data_restaurant = response_restaurant.json()
            results_restaurant.append(data_restaurant)
        except json.JSONDecodeError: pass

    # Define parameters for the API request for this location query: bar
    params_bar = {
        'll': f'{latitude},{longitude}',
        'radius': 1000,
        'query': 'bar'
    }

    # Get requests for params_bar
    response_bar = requests.get(url, params=params_bar, headers=headers)

    # Check the response status code and add values to results list if exists
    if response_bar.status_code == 200:
        try: 
            data_bar = response_bar.json()
            results_bar.append(data_bar)
        except json.JSONDecodeError: pass

    # Define parameters for the API request for this location query: performing arts venue
    params_art = {
        'll': f'{latitude},{longitude}',
        'radius': 1000,
        'query': 'Performing Arts Venue'
    }

    # Get requests for params_art
    response_art = requests.get(url, params=params_art, headers=headers)
    
    # Check the response status code and add values to results list if exists
    if response_art.status_code == 200:
        try: 
            data_art = response_art.json()
            results_art.append(data_art)
        except json.JSONDecodeError: pass            



In [72]:
results_bar[0]['results'][0]['location']['postcode']

'M4L 1A8'

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)
Put your parsed results into a DataFrame

In [79]:
normalize_results_foursquare_df = pd.DataFrame(columns= ['name',
  # 'postalcode', 
  'distance', 'latitude', 'longitude', 'api', 'query']
)

def normalize_results_foursquare (results_list, query):
  """
  Function to return the query results from the json hierarchy to a flattened dataframe
  """
  global normalize_results_foursquare_df
  global results_list_indx_results_df

  # Iterate through each of the results of the query
  for indx_all_addresses_at_bike_station in range(len(results_list)):

    # Create dataframe of context information
    results_list_indx_context_df = pd.DataFrame(results_list[indx_all_addresses_at_bike_station]['context'])

    # Extract API request parameters within context
    context_latitude = results_list_indx_context_df['geo_bounds']['circle']['center']['latitude']
    context_longitude = results_list_indx_context_df['geo_bounds']['circle']['center']['longitude']
    context_radius = results_list_indx_context_df['geo_bounds']['circle']['radius']

    for indx_single_address_at_bike_station in range(len(results_list[indx_all_addresses_at_bike_station]['results'])):

      # Create a DataFrame matching the length of the results dictionary
      results_list_indx_dict = {
        'name':             [results_list[indx_all_addresses_at_bike_station]['results'][indx_single_address_at_bike_station]['name']], 
        # 'postalcode':       [results_list[indx_all_addresses_at_bike_station]['results'][indx_single_address_at_bike_station]['location']['postcode']],
        'distance':         [results_list[indx_all_addresses_at_bike_station]['results'][indx_single_address_at_bike_station]['distance']],
        'latitude': [context_latitude], 
        'longitude': [context_longitude], 
        'api': 'foursquare',
        'query': [query]
      }
      results_list_indx_df = pd.DataFrame(results_list_indx_dict)

      # Append to 
      normalize_results_foursquare_df = normalize_results_foursquare_df.append(results_list_indx_df, ignore_index=True)


In [80]:
# Run the normalized results
normalize_results_foursquare (results_bar, 'bar')
normalize_results_foursquare (results_art, 'art')
normalize_results_foursquare (results_restaurant, 'restaurant')
normalize_results_foursquare_df.to_csv(r'C:\Users\akans\Documents\Kierans_Documents\LHL\Statistical-Modelling-Project\data\results_four.csv')

In [97]:

# results_bar_df_normalized = pd.DataFrame(columns=[
#     'fsq_id', 'chains', 'distance', 'link', 'name', 'related_places',
#     'timezone', 'category_id', 'category_name', 'category_icon.prefix',
#     'category_icon.suffix', 'geocode_drop_off.latitude',
#     'geocode_drop_off.longitude', 'geocode_main.latitude',
#     'geocode_main.longitude', 'geocode_roof.latitude',
#     'geocode_roof.longitude', 'location_address', 'location_country',
#     'location_cross_street', 'location_formatted_address',
#     'location_locality', 'location_postcode', 'location_region',
#     'context_latitude', 'context_longitude', 'context_radius', 'api', 'query']
# )

# for list_value in range(len(results_bar)):
    
#   data = results_bar[list_value]

#   # Create a DataFrame from the list of dictionaries
#   results_bar_row_df = pd.DataFrame(data['results'])

#   # Explode the 'categories' column to create multiple rows, making 'id' and 'name' part of the primary key along with 'fsq_id'
#   results_bar_row_df = results_bar_row_df.explode('categories')
#   results_bar_row_df = results_bar_row_df.reset_index(drop=True)

#   # Normalize the columns containing dictionaries
#   df_categories = pd.json_normalize(results_bar_row_df['categories']).add_prefix('category_')
#   df_geocodes = pd.json_normalize(results_bar_row_df['geocodes']).add_prefix('geocode_')
#   df_location = pd.json_normalize(results_bar_row_df['location']).add_prefix('location_')

#   # Drop the original columns containing dictionaires
#   results_bar_row_df = results_bar_row_df.drop(['categories', 'geocodes', 'location'], axis=1)

#   # Concatenate the normalized columns of dictionaries with the original DataFrame
#   results_bar_row_df = pd.concat([results_bar_row_df, df_categories, df_geocodes, df_location], axis=1)

#   # Create dataframe of context information
#   results_bar_row_df2 = pd.DataFrame(data['context'])

#   # Extract API request parameters
#   context_latitude = results_bar_row_df2['geo_bounds']['circle']['center']['latitude']
#   context_longitude = results_bar_row_df2['geo_bounds']['circle']['center']['longitude']
#   context_radius = results_bar_row_df2['geo_bounds']['circle']['radius']

#   # Create a DataFrame matching the length of the results dictionary
#   results_bar_row_df2 = {
#     'context_latitude': [context_latitude] * len(results_bar_row_df), 
#     'context_longitude': [context_longitude] * len(results_bar_row_df), 
#     'context_radius': [context_radius] * len(results_bar_row_df),
#     'api': 'foursquare' * len(results_bar_row_df),
#     'query': 'bar' * len(results_bar_row_df)
#   }
#   results_bar_row_df2 = pd.DataFrame(results_bar_row_df2)

#   # Concatenate
#   resulting_df = pd.concat([results_bar_row_df, results_bar_row_df2], axis = 1)

#   # Append to 
#   results_bar_df_normalized = results_bar_df_normalized.append(resulting_df, ignore_index=True)
  

# # Display the DataFrame
# # print(resulting_df)
# results_bar_df_normalized

,fsq_id,chains,distance,link,name,related_places,timezone,category_id,category_name,category_icon.prefix,...,location_region,context_latitude,context_longitude,context_radius,api,query,location_address_extended,geocode_front_door.latitude,geocode_front_door.longitude,location_po_box
0,58d4650e2f91cb5ccec03447,[],194,/v3/places/58d4650e2f91cb5ccec03447,Rorschach Brewing,{},America/Toronto,13029,Brewery,https://ss3.4sqi.net/img/categories_v2/food/br...,...,ON,43.665269,-79.319796,1000,foursquarefoursquarefoursquarefoursquarefoursq...,barbarbarbarbarbarbarbarbarbarbarbarbarbarbarb...,NaN,NaN,NaN,NaN
1,58d4650e2f91cb5ccec03447,[],194,/v3/places/58d4650e2f91cb5ccec03447,Rorschach Brewing,{},America/Toronto,13065,Restaurant,https://ss3.4sqi.net/img/categories_v2/food/de...,...,ON,43.665269,-79.319796,1000,foursquarefoursquarefoursquarefoursquarefoursq...,barbarbarbarbarbarbarbarbarbarbarbarbarbarbarb...,NaN,NaN,NaN,NaN
2,5b3af4f125fb7b0039f9f588,[],842,/v3/places/5b3af4f125fb7b0039f9f588,Black Lab Brewing,{},America/Toronto,13029,Brewery,https://ss3.4sqi.net/img/categories_v2/food/br...,...,ON,43.665269,-79.319796,1000,foursquarefoursquarefoursquarefoursquarefoursq...,barbarbarbarbarbarbarbarbarbarbarbarbarbarbarb...,NaN,NaN,NaN,NaN
3,5957bd26d1a40251f53875e5,[],804,/v3/places/5957bd26d1a40251f53875e5,Godspeed Brewery,{},America/Toronto,13003,Bar,https://ss3.4sqi.net/img/categories_v2/nightli...,...,ON,43.665269,-79.319796,1000,foursquarefoursquarefoursquarefoursquarefoursq...,barbarbarbarbarbarbarbarbarbarbarbarbarbarbarb...,NaN,NaN,NaN,NaN
4,5957bd26d1a40251f53875e5,[],804,/v3/places/5957bd26d1a40251f53875e5,Godspeed Brewery,{},America/Toronto,13029,Brewery,https://ss3.4sqi.net/img/categories_v2/food/br...,...,ON,43.665269,-79.319796,1000,foursquarefoursquarefoursquarefoursquarefoursq...,barbarbarbarbarbarbarbarbarbarbarbarbarbarbarb...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13978,51795105498e74dbd890c160,[],80,/v3/places/51795105498e74dbd890c160,Burrito Place,{},America/Toronto,13305,Burrito Restaurant,https://ss3.4sqi.net/img/categories_v2/food/bu...,...,ON,43.781877,-79.415285,1000,foursquarefoursquarefoursquarefoursquarefoursq...,barbarbarbarbarbarbarbarbarbarbarbarbarbarbarbar,NaN,NaN,NaN,NaN
13979,4bb6caafef159c746d8e76f7,[],280,/v3/places/4bb6caafef159c746d8e76f7,Twister Karaoke,{},America/Toronto,13015,Karaoke Bar,https://ss3.4sqi.net/img/categories_v2/nightli...,...,ON,43.781877,-79.415285,1000,foursquarefoursquarefoursquarefoursquarefoursq...,barbarbarbarbarbarbarbarbarbarbarbarbarbarbarbar,at Finch Ave,NaN,NaN,NaN
13980,4bb6caafef159c746d8e76f7,[],280,/v3/places/4bb6caafef159c746d8e76f7,Twister Karaoke,{},America/Toronto,10000,Arts and Entertainment,https://ss3.4sqi.net/img/categories_v2/arts_en...,...,ON,43.781877,-79.415285,1000,foursquarefoursquarefoursquarefoursquarefoursq...,barbarbarbarbarbarbarbarbarbarbarbarbarbarbarbar,at Finch Ave,NaN,NaN,NaN
13981,5ac6b2e2efa82a057cbd0c49,[],617,/v3/places/5ac6b2e2efa82a057cbd0c49,The Alley,{},America/Toronto,13033,Bubble Tea Shop,https://ss3.4sqi.net/img/categories_v2/food/bu...,...,ON,43.781877,-79.415285,1000,foursquarefoursquarefoursquarefoursquarefoursq...,barbarbarbarbarbarbarbarbarbarbarbarbarbarbarbar,NaN,NaN,NaN,NaN


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [8]:
# Assign Environment Variables
client_id = os.environ["YELP_CLIENT_ID"]
api_key = os.environ["YELP_API_KEY"]

# Define the endpoint URL
url = "https://api.yelp.com/v3/businesses/search"

# Create an empty list to store the results
results_yelp_restaurant = []
results_yelp_bar = []
results_yelp_art = []

# import city_bikes list of latitude and longitude
bike_stations_toronto = pd.read_csv(r'C:\Users\akans\Documents\Kierans_Documents\LHL\Statistical-Modelling-Project\data\bike_stations_toronto.csv')

# Define Headers for the API requests
headers = {
    'Authorization': f'Bearer {api_key}',
}


# Iterate through the DataFrame rows
for index, row in bike_stations_toronto.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    # Define parameters for the API request for this location query: restaurant
    params_restaurant = {
        'll': f'{latitude},{longitude}',
        'radius': 1000,
        'term': 'restaurant'
    }

    # Get requests for params_restaurant
    response_yelp_restaurant = requests.get(url, params=params_restaurant, headers=headers)

    # Check the response status code and add values to results list if exists
    if response_yelp_restaurant.status_code == 200:
        try: 
            data_restaurant = response_yelp_restaurant.json()
            results_yelp_restaurant.append(data_restaurant)
        except json.JSONDecodeError: pass

    # Define parameters for the API request for this location query: bar
    params_bar = {
        'll': f'{latitude},{longitude}',
        'radius': 1000,
        'term': 'bar'
    }

    # Get requests for params_bar
    response_yelp_bar = requests.get(url, params=params_bar, headers=headers)

    # Check the response status code and add values to results list if exists
    if response_yelp_bar.status_code == 200:
        try: 
            data_bar = response_yelp_bar.json()
            results_yelp_bar.append(data_bar)
        except json.JSONDecodeError: pass

    # Define parameters for the API request for this location query: performing arts venue
    results_yelp_art = {
        'll': f'{latitude},{longitude}',
        'radius': 1000,
        'term': 'Performing Arts Venue'
    }

    # Get requests for params_art
    response_yelp_art = requests.get(url, params=params_art, headers=headers)
    
    # Check the response status code and add values to results list if exists
    if response_yelp_art.status_code == 200:
        try: 
            data_art = response_yelp_art.json()
            results_yelp_art.append(data_art)
        except json.JSONDecodeError: pass            

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

{'ll': '43.657343,-79.43111999999999',
 'radius': 1000,
 'term': 'Performing Arts Venue'}

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating